In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('NYPD_Data.csv')
df.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,ADDR_PCT_CD,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,394278393,1/1/15,9:00:00,4/15/16,8:30:00,4/15/16,340,FRAUDS,718.0,"FRAUD,UNCLASSIFIED-MISDEMEANOR",...,41,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1015226.0,239233.0,40.823269,-73.888081,"(40.823268829, -73.888081)"
1,722254800,1/1/15,0:01:00,6/11/16,20:15:00,4/10/16,116,SEX CRIMES,168.0,SODOMY 1,...,77,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,997245.0,204130.0,40.726965,-73.953117,"(40.726965067, -73.953116629)"
2,530381050,1/1/15,0:00:00,4/2/16,14:00:00,4/2/16,578,HARRASSMENT 2,638.0,"HARASSMENT,SUBD 3,4,5",...,47,INSIDE,RESIDENCE - APT. HOUSE,NaN,NaN,1023381.0,256426.0,40.870426,-73.858516,"(40.870426138, -73.858515601)"
3,429186741,1/1/15,12:00:00,3/16/16,16:00:00,3/29/16,112,THEFT-FRAUD,739.0,"FRAUD,UNCLASSIFIED-FELONY",...,50,INSIDE,RESIDENCE - PUBLIC HOUSING,NaN,FORT INDEPENDENCE STREET-HEATH AVENUE,1011678.0,259907.0,40.880025,-73.900816,"(40.880024527, -73.900815814)"
4,388921549,1/1/15,0:01:00,5/30/16,19:29:00,3/28/16,340,FRAUDS,718.0,"FRAUD,UNCLASSIFIED-MISDEMEANOR",...,88,INSIDE,RESIDENCE - PUBLIC HOUSING,ROBERT E. VENABLE PARK,INGERSOLL,989561.0,192267.0,40.694412,-73.980849,"(40.694411894, -73.980849181)"


In [ ]:
df_=df[['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'ADDR_PCT_CD']]

df__ = pd.to_datetime(df_['CMPLNT_FR_DT'])
df_weekdays = df__.dt.dayofweek
df_['CMPLNT_FR_DT'] = df_weekdays
df____ = pd.to_datetime(df_['CMPLNT_FR_TM']).dt.hour
df_['CMPLNT_FR_TM'] = df____[:]/6
df_['CMPLNT_FR_TM'].astype(int)
Day_num = 67
Training_day = 54.0
Testing_day = 13.0

In [65]:
df_.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,ADDR_PCT_CD
0,394278393,3,1.5,41
1,722254800,3,0.0,77
2,530381050,3,0.0,47
3,429186741,3,2.0,50
4,388921549,3,0.0,88


In [62]:
train, test = train_test_split(df_, test_size = 0.194)

X1 = []
X2 = []
X3 = []
Y_test = []
Y_train = []

for day in range(7):
    for hour in range(4):
        for PCT in range(34):
            X1.append(day)
            X2.append(hour)
            X3.append(PCT)
            Y_test.append(0.0)
            Y_train.append(0.0)
            
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
X = np.transpose(np.array([X1,X2,X3]))            
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

def calculate_frequency(df_,Num_day,Y):
    CrimeNum = df_.shape[0]
    for i in range(CrimeNum):
        if df_.iloc[i,3] <= 34:
            day_ = df_.iloc[i,1]
            hour_ = int(df_.iloc[i,2])
            #print hour_
            #print type(hour_)
            PCT_ = df_.iloc[i,3]
            Y[int(day_*4*34+hour_*34+PCT_-1)] += 1.0
    Y = Y/Num_day
    return Y

Y_train = calculate_frequency(train,Training_day,Y_train)
Y_test = calculate_frequency(test,Testing_day,Y_test)

In [63]:
from sklearn.ensemble import RandomForestRegressor
CLS = RandomForestRegressor(max_depth=100,n_estimators=50)
CLS.fit(X,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [64]:
print CLS.score(X,Y_test)

0.854391785279
